<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/prepareTransactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy_financial
!pip install pyxirr

In [2]:
import pandas as pd
import numpy as np
import yfinance as yfin
from datetime import date, timedelta
import numpy_financial as npf
from pyxirr import xirr

from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

from utilities import read_csv
import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# function to consolidate and return transactions
def prepare_transactions(myts_5ps, myts_zrd, mypfs_df):

  mypfs = mypfs_df[mypfs_df['InPortfolio'].isin(['SV','DM'])]['Symbol']

  myts_zrd['amount'] = myts_zrd['quantity'] * myts_zrd['price']
  myts_zrd_grp = myts_zrd.groupby(['trade_date', 'symbol', 'trade_type'])[['quantity', 'amount']].sum().reset_index()
  myts_zrd_grp['trade_date'] = pd.to_datetime(myts_zrd_grp['trade_date'], format='%d-%m-%Y').dt.strftime('%Y-%m-%d')

  myts_5ps = myts_5ps.rename(columns={'Transaction Date': 'trade_date', 'Symbol': 'symbol', 'Type': 'trade_type', 'Quantity': 'quantity', 'Price': 'price'})
  columns = ['trade_date', 'symbol', 'trade_type', 'quantity', 'price']
  myts_5ps = myts_5ps[columns]
  myts_5ps['amount'] = myts_5ps['quantity'] * myts_5ps['price']
  myts_5ps = myts_5ps.groupby(['trade_date', 'symbol', 'trade_type'])[['quantity', 'amount']].sum().reset_index()
  myts_5ps.loc[myts_5ps['trade_type'] == 'Buy', 'trade_type'] = 'buy'
  myts_5ps.loc[myts_5ps['trade_type'] == 'Sell', 'trade_type'] = 'sell'
  myts_5ps['trade_date'] = pd.to_datetime(myts_5ps['trade_date'], format='%b %d %Y').dt.strftime('%Y-%m-%d')

  df_myts = pd.concat([myts_zrd_grp, myts_5ps], ignore_index=True)
  df_myts.loc[df_myts['trade_type'] == 'buy', 'amount'] *= -1
  df_myts.loc[df_myts['trade_type'] == 'sell', 'quantity'] *= -1

  return df_myts

# function to get current value of stocks
def get_curr_trade(stock_symbol, quantity):

  stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='1d', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
  curr_date = (date.today() + timedelta(days=1)).strftime('%Y-%m-%d')
  price = round(stock_df['Close'].values[0],2)
  amount = round(quantity * price, 2)

  return   pd.DataFrame.from_dict({'trade_date': [curr_date],
                          'symbol': [stock_symbol],
                          'trade_type':['sell'],
                          'quantity': [quantity],
                          'amount': [amount]})

# module to update my prospects
def update_my_prospects(df_xirr):
  mypps_df = read_csv('myProspectsScrips.csv')

  # Merge the two DataFrames on the common column
  merged_df = pd.merge(mypps_df, df_xirr, on='Symbol', how='left')

  # Update the column in the CSV DataFrame with the values from the second DataFrame
  mypps_df['XIRR'] = merged_df['XIRR%']
  mypps_df.loc[mypps_df['Symbol'].isin(mypfs), 'InFolio'] = 1

  # Save the updated DataFrame back to a CSV file
  mypps_df.to_csv('data/myProspectsScrips.csv', index=False)
  df_xirr.to_csv('data/myStocks-XIRR.csv', index=False)

# Define function to calculate XIRR
def calculate_xirr(cash_flows):
    #print(cash_flows)
    dates = pd.to_datetime(cash_flows['trade_date'])
    amounts = cash_flows['amount']

    # Check if both negative and positive amounts exist
    if (amounts < 0).any() and (amounts > 0).any():
        return xirr(pd.DataFrame({"dates": dates, "amounts": amounts}))
    else:
        return np.nan  # Return NaN if not both types of cash flows are present

# Define function to calculate XIRR
def calculate_abs(cash_flows):
    #print(cash_flows)
    dates = pd.to_datetime(cash_flows['trade_date'])
    amounts = cash_flows['amount']

    # Check if both negative and positive amounts exist
    invested = round(-sum(cash_flows[cash_flows['trade_type'] == 'buy']['amount']),0)
    compounded = round(sum(cash_flows[cash_flows['trade_type'] == 'sell']['amount']),0)

    if invested == 0:
      return np.nan  # Or any other appropriate value to indicate this situation
    else:
      abs = (compounded - invested)/invested

      return abs

In [4]:
myts_5ps = read_csv('myTransactions_5Paisa.csv')
myts_zrd = read_csv('myTransactions_Zerodha.csv')
mypfs_df = read_csv('myPortfolioStocks.csv')

df_myts = prepare_transactions(myts_5ps, myts_zrd, mypfs_df)

tmp_df = df_myts.groupby(['symbol'])[['quantity']].sum().reset_index()
tmp_df = tmp_df[tmp_df['quantity'] > 0]

for index, row in tmp_df.iterrows():
  stock_symbol = row['symbol']
  quantity = row['quantity']
  tmp_df = get_curr_trade(stock_symbol, quantity)
  df_myts = pd.concat([df_myts, tmp_df], ignore_index=True)

df = df_myts

# Group by 'symbol' and calculate XIRR and CAGR
df_xirr = df.groupby('symbol').apply(lambda x: pd.Series({
    'XIRR%': round(calculate_xirr(x)*100,0),
     'ABS%': round(calculate_abs(x)*100,0)
})).reset_index()

# Print results
print('-----------')
print('CAGR%: ', round(calculate_xirr(df_myts)*100,2))
print('-----------')

  #print(get_curr_trade(stock_symbol, quantity))

-----------
CAGR%:  32.08
-----------


In [5]:
mypfs = mypfs_df[mypfs_df['InPortfolio'].isin(['SV','DM'])]['Symbol']
df_xirr['InPF'] = 0
df_xirr.loc[df_xirr['symbol'].isin(mypfs), 'InPF'] = 1
df_xirr = df_xirr.rename(columns={'symbol': 'Symbol'})
update_my_prospects(df_xirr)

df_xirr[df_xirr['InPF']==0].sort_values(by = 'XIRR%', ascending=False)

,Symbol,XIRR%,ABS%,InPF
26,CASTROLIND,2824.0,30.0,0
49,HINDPETRO,1817.0,34.0,0
65,JSWSTEEL,802.0,29.0,0
61,J&KBANK,680.0,35.0,0
81,NMDC,507.0,41.0,0
78,NIACL,442.0,100.0,0
13,BAJAJHIND,431.0,19.0,0
22,BIRLAMONEY,365.0,55.0,0
27,CDSL,271.0,73.0,0
84,PNB,238.0,31.0,0
